In [5]:
import src.utilities.utils as utils 
test_data = utils.read_jsonl('data/snli_1.0/snli_1.0_test.jsonl')
train_data = utils.read_jsonl('data/snli_1.0/snli_1.0_train.jsonl')
dev_data = utils.read_jsonl('data/snli_1.0/snli_1.0_dev.jsonl')

In [6]:
# nums = ['2','3','4','5','6','7','8','9','10']
# special_numericals = ['some', 'many', 'all'] # TODO
# number_words = utils.plural_words_for_numbers + nums

convert_dict_plural = {
    'two':2,
    'three':3,
    'four':4,
    'five':5,
    'six':6,
    'seven':7,
    'eight':8,
    'nine':9,
    'ten':10,
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10
}

def numerical_word_in_sentence(sentence):
    return [x for x in sentence.split() if x in convert_dict_plural]

def subset_with_numericals(dataset):
    result = []
    for data in dataset:
        sent1_num_words = numerical_word_in_sentence(data['sentence1'])
        sent2_num_words = numerical_word_in_sentence(data['sentence2'])

        if sent1_num_words or sent2_num_words:
#             print((sent1_num_words, sent2_num_words))
#             print(data['gold_label'])
#             print(data['sentence1'])
#             print(data['sentence2'])
#             print()
            result.append(data)
    return result

def subset_with_same_numericals(dataset):
    result = []
    for data in dataset:
        sent1_num_words = numerical_word_in_sentence(data['sentence1'])
        sent2_num_words = numerical_word_in_sentence(data['sentence2'])

        if sent1_num_words == sent2_num_words and len(sent1_num_words) == 1: # TODO: When len(list) > 1
#             print((sent1_num_words, sent2_num_words))
#             print(data['gold_label'])
#             print(data['sentence1'])
#             print(data['sentence2'])
#             print()
            result.append(data)
    return result

In [7]:
def subset_by_relationship_type(dataset, relation):
    result = []
    return [data for data in dataset if data['gold_label'] == relation]

In [8]:
test_data_entailment = subset_by_relationship_type(test_data, 'entailment')
train_data_entailment = subset_by_relationship_type(train_data, 'entailment')
dev_data_entailment = subset_by_relationship_type(dev_data, 'entailment')
# test_data_contradiction = subset_by_relationship_type(test_data, 'contradiction')
# test_data_neutral = subset_by_relationship_type(test_data, 'neutral')

In [9]:
len(test_data_entailment)

3368

In [10]:
len(train_data_entailment)

183416

In [11]:
len(dev_data_entailment)

3329

In [12]:
# Entailment pairs that include numerical words in either one of the sentences
test_entailment_with_numericals = subset_with_numericals(test_data_entailment)
train_entailment_with_numericals = subset_with_numericals(train_data_entailment)
dev_entailment_with_numericals = subset_with_numericals(dev_data_entailment)

In [13]:
# test_entailment_with_numericals

In [14]:
# Entailment pairs that include equal numerical words
test_entailment_with_same_numericals = subset_with_same_numericals(test_entailment_with_numericals)
train_entailment_with_same_numericals = subset_with_same_numericals(train_entailment_with_numericals)
dev_entailment_with_same_numericals = subset_with_same_numericals(dev_entailment_with_numericals)

In [15]:
# test_entailment_with_same_numericals

In [16]:
def get_binary_pairs(list_of_dic):
    res = []
    for each in list_of_dic:
        res.append(
            (each['sentence1_binary_parse'], each['sentence2_binary_parse'])
        )
    return res

In [17]:
test_binary_entailment_same_example = get_binary_pairs(test_entailment_with_same_numericals)
train_binary_entailment_same_example = get_binary_pairs(train_entailment_with_same_numericals)
dev_binary_entailment_same_example = get_binary_pairs(dev_entailment_with_same_numericals)

test_binary_entailment_same_example

[('( ( ( A man ) ( ( standing ( in ( front ( of ( ( a building ) ( on ( ( the phone ) ( as ( two men ) ) ) ) ) ) ) ) ) ( to ( ( the ( side pain ) ) ( on ( the side ) ) ) ) ) ) . )',
  '( ( ( a guy ) ( near ( a building ) ) ) ( stands ( by ( two ( other men ) ) ) ) )'),
 ('( ( ( ( A crowd ) ( of people ) ) ( ( looking up ) ( at ( ( 3 people ) ( on ( ( the edge ) ( of ( ( the roof ) ( of ( a building ) ) ) ) ) ) ) ) ) ) . )',
  "( ( ( The crowd ) ( on ( the ground ) ) ) ( ( is ( watching ( ( 3 people ) ( on ( ( the ( roof 's ) ) edge ) ) ) ) ) . ) )"),
 ('( ( ( Six or ) ( seven people ) ) ( ( are ( standing ( on ( ( ( ( a pier ) ( with ( a table ) ) ) and ) ( ( a pair ) ( of ( glasses ( in ( the foreground ) ) ) ) ) ) ) ) ) . ) )',
  '( ( ( Six or ) ( seven people ) ) ( ( are ( standing ( on ( a pier ) ) ) ) . ) )'),
 ('( ( A boy ) ( ( is ( ( riding down ) ( ( the road ) ( between ( two cows ) ) ) ) ) . ) )',
  '( ( ( ( A boy ) , ) ( ( ( a road ) and ) ( two cows ) ) ) . )'),
 ('( ( The 

In [18]:
numerical_word_in_sentence(test_binary_entailment_same_example[0][0])[0]

'two'

## 1. Generate Entailment from Entailment

num1 == num2 $\Longrightarrow$ num1 == num2

In [138]:
def generate_same_to_same_plural(binary_list, label):
    res = []
    for i in range(len(binary_list)):
        orig_rem_bin_pair = utils.remove_parenthesis(binary_list[i])
        new_bin_pairs = utils.same_to_same_plural_number(
            binary_list[i],
            numerical_word_in_sentence(binary_list[i][0])[0]
        )
        for pair in new_bin_pairs:
            rem_bin_pair = utils.remove_parenthesis(pair)
            tmp_dict = dict()
            tmp_dict['gold_label'] = label
            tmp_dict['sentence1'] = rem_bin_pair[0]            
            tmp_dict['sentence2'] = rem_bin_pair[1]            
            tmp_dict['sentence1_binary_parse'] = pair[0]            
            tmp_dict['sentence2_binary_parse'] = pair[1]   
            tmp_dict['orig_sentence1'] = orig_rem_bin_pair[0]
            tmp_dict['orig_sentence2'] = orig_rem_bin_pair[1]
            tmp_dict['orig_sentence1_binary_parse'] = binary_list[i][0]
            tmp_dict['orig_sentence2_binary_parse'] = binary_list[i][1]
            res.append(tmp_dict)
    return res

In [135]:
test_entailment_same_to_same_plural = generate_same_to_same_plural(test_binary_entailment_same_example, 'entailment')
train_entailment_same_to_same_plural = generate_same_to_same_plural(train_binary_entailment_same_example, 'entailment')
dev_entailment_same_to_same_plural = generate_same_to_same_plural(dev_binary_entailment_same_example, 'entailment')

In [137]:
import json
with open('test_entailment_same_to_same_plural.json', 'w') as outfile:
    json.dump(test_entailment_same_to_same_plural, outfile)

with open('train_entailment_same_to_same_plural.json', 'w') as outfile:
    json.dump(train_entailment_same_to_same_plural, outfile)
    
with open('dev_entailment_same_to_same_plural.json', 'w') as outfile:
    json.dump(dev_entailment_same_to_same_plural, outfile)

In [136]:
print(len(test_entailment_same_to_same_plural))
print(len(train_entailment_same_to_same_plural))
print(len(dev_entailment_same_to_same_plural))

896
48480
1056


In [172]:
test_entailment_same_to_same_plural[:20:4]

[{'gold_label': 'entailment',
  'sentence1': 'A man standing in front of a building on the phone as 3 men to the side pain on the side .',
  'sentence2': 'a guy near a building stands by 3 other men',
  'sentence1_binary_parse': '( ( ( A man ) ( ( standing ( in ( front ( of ( ( a building ) ( on ( ( the phone ) ( as ( 3 men ) ) ) ) ) ) ) ) ) ( to ( ( the ( side pain ) ) ( on ( the side ) ) ) ) ) ) . )',
  'sentence2_binary_parse': '( ( ( a guy ) ( near ( a building ) ) ) ( stands ( by ( 3 ( other men ) ) ) ) )',
  'orig_sentence1': 'A man standing in front of a building on the phone as two men to the side pain on the side .',
  'orig_sentence2': 'a guy near a building stands by two other men',
  'orig_sentence1_binary_parse': '( ( ( A man ) ( ( standing ( in ( front ( of ( ( a building ) ( on ( ( the phone ) ( as ( two men ) ) ) ) ) ) ) ) ) ( to ( ( the ( side pain ) ) ( on ( the side ) ) ) ) ) ) . )',
  'orig_sentence2_binary_parse': '( ( ( a guy ) ( near ( a building ) ) ) ( stands (

## 2. Generate Neutral from Entailment

num1 == num2 $\Longrightarrow$ num1 < num2

In [159]:
def generate_same_to_different_plural(binary_list, label):
    res = []
    for i in range(len(binary_list)):
        orig_rem_bin_pair = utils.remove_parenthesis(binary_list[i])
        new_bin_pairs = utils.same_to_different_plural_number(
            binary_list[i],
            numerical_word_in_sentence(binary_list[i][0])[0],
            False
        )
        for pair in new_bin_pairs:
            rem_bin_pair = utils.remove_parenthesis(pair)
            tmp_dict = dict()
            tmp_dict['gold_label'] = label
            tmp_dict['sentence1'] = rem_bin_pair[0]
            tmp_dict['sentence2'] = rem_bin_pair[1]
            tmp_dict['sentence1_binary_parse'] = pair[0]
            tmp_dict['sentence2_binary_parse'] = pair[1]      
            tmp_dict['orig_sentence1'] = orig_rem_bin_pair[0]
            tmp_dict['orig_sentence2'] = orig_rem_bin_pair[1]
            tmp_dict['orig_sentence1_binary_parse'] = binary_list[i][0]
            tmp_dict['orig_sentence2_binary_parse'] = binary_list[i][1]            
            res.append(tmp_dict)
    return res

In [160]:
test_neutral_same_to_different_plural = generate_same_to_different_plural(test_binary_entailment_same_example, 'neutral')
train_neutral_same_to_different_plural = generate_same_to_different_plural(train_binary_entailment_same_example, 'neutral')
dev_neutral_same_to_different_plural = generate_same_to_different_plural(dev_binary_entailment_same_example, 'neutral')


In [166]:
import json
with open('test_neutral_same_to_different_plural.json', 'w') as outfile:
    json.dump(test_neutral_same_to_different_plural, outfile)

with open('train_neutral_same_to_different_plural.json', 'w') as outfile:
    json.dump(train_neutral_same_to_different_plural, outfile)
    
with open('dev_neutral_same_to_different_plural.json', 'w') as outfile:
    json.dump(dev_neutral_same_to_different_plural, outfile)

In [170]:
print(len(test_neutral_same_to_different_plural))
print(len(train_neutral_same_to_different_plural))
print(len(dev_neutral_same_to_different_plural))

4032
218160
4752


In [171]:
test_neutral_same_to_different_plural[:20:4]

[{'gold_label': 'neutral',
  'sentence1': 'A man standing in front of a building on the phone as 2 men to the side pain on the side .',
  'sentence2': 'a guy near a building stands by 3 other men',
  'sentence1_binary_parse': '( ( ( A man ) ( ( standing ( in ( front ( of ( ( a building ) ( on ( ( the phone ) ( as ( 2 men ) ) ) ) ) ) ) ) ) ( to ( ( the ( side pain ) ) ( on ( the side ) ) ) ) ) ) . )',
  'sentence2_binary_parse': '( ( ( a guy ) ( near ( a building ) ) ) ( stands ( by ( 3 ( other men ) ) ) ) )',
  'orig_sentence1': 'A man standing in front of a building on the phone as two men to the side pain on the side .',
  'orig_sentence2': 'a guy near a building stands by two other men',
  'orig_sentence1_binary_parse': '( ( ( A man ) ( ( standing ( in ( front ( of ( ( a building ) ( on ( ( the phone ) ( as ( two men ) ) ) ) ) ) ) ) ) ( to ( ( the ( side pain ) ) ( on ( the side ) ) ) ) ) ) . )',
  'orig_sentence2_binary_parse': '( ( ( a guy ) ( near ( a building ) ) ) ( stands ( by

## 3. Generate Contradiction from Entailment
subject1 == subect2 $\Longrightarrow$ subject1 $\leftrightarrow$ subect2

In [111]:
# Find the word to replace to an antonym word. Pattern: the word after numerical word
def return_next_word(word, sentence):
    list_of_words = sentence.split()
    if list_of_words.index(word) + 1 < len(list_of_words):
        return list_of_words[list_of_words.index(word) + 1]
    else:
        raise Exception('{} is the last word in the sentence'.format(word))

noun_to_replace = return_next_word("two", "There are two boys")
print(noun_to_replace)

boys


In [173]:
from nltk.corpus import wordnet
from pattern.en import pluralize

# Return the antonym of input word. Make them plural # TODO: pluralize doesn't make sense when the antonym is an adjective
def return_antonym_plural(word):
    antonyms = []
    for syn in wordnet.synsets(word):
        for lm in syn.lemmas():
            if lm.antonyms():
                antonyms.append(lm.antonyms()[0].name())
                
    if antonyms:
        antonyms.sort(key=lambda item: (len(item), item))
        return pluralize(antonyms[0])
    else:
        print('Antonym of {} could not be found.'.format(word))
        return None

return_antonym_plural('boys')

'girls'

In [175]:
def generate_same_to_different_plural(binary_list):
    res = []
    for i in range(len(binary_list)):
        sentence1_bin = binary_list[i][0]
        sentence2_bin = binary_list[i][1]
        sentence1, sentence2 = utils.remove_parenthesis(binary_list[i])
        numerical_word = numerical_word_in_sentence(binary_list[i][0])[0]
        noun_to_replace = return_next_word(numerical_word, sentence2) # subject to be replaced in sentence2
        ant_noun_to_replace = return_antonym_plural(noun_to_replace) # antonym of the subject
        if ant_noun_to_replace:
            new_sentence2_bin = utils.return_sent_words_replaced_case_sensitive(
                sentence2_bin, 
                noun_to_replace, 
                ant_noun_to_replace
            )
            new_sentence2 = utils.return_sent_words_replaced_case_sensitive(
                sentence2, 
                noun_to_replace, 
                ant_noun_to_replace
            )            

            tmp_dict = dict()
            tmp_dict['gold_label'] = 'neutral'
            tmp_dict['sentence1'] = sentence1
            tmp_dict['sentence2'] = new_sentence2
            tmp_dict['sentence1_binary_parse'] = binary_list[i][0]
            tmp_dict['sentence2_binary_parse'] = new_sentence2_bin
            tmp_dict['orig_sentence1'] = sentence1
            tmp_dict['orig_sentence2'] = sentence2
            tmp_dict['orig_sentence1_binary_parse'] = binary_list[i][0]
            tmp_dict['orig_sentence2_binary_parse'] = binary_list[i][1]  
            res.append(tmp_dict)
        
    return res

In [176]:
res = generate_same_to_different_plural(test_binary_entailment_same_example)

Antonym of people could not be found.
Antonym of people could not be found.
Antonym of cows could not be found.
Antonym of wheeler could not be found.
Antonym of people could not be found.
Antonym of players could not be found.
Antonym of windows could not be found.
Antonym of brooms could not be found.
Antonym of cameras could not be found.
Antonym of smiling could not be found.
Antonym of boating could not be found.
Antonym of dogs could not be found.
Antonym of people could not be found.
Antonym of dogs could not be found.
Antonym of boats could not be found.
Antonym of people could not be found.


In [179]:
res[:5]

[{'gold_label': 'neutral',
  'sentence1': 'A man standing in front of a building on the phone as two men to the side pain on the side .',
  'sentence2': 'a guy near a building stands by two sames men',
  'sentence1_binary_parse': '( ( ( A man ) ( ( standing ( in ( front ( of ( ( a building ) ( on ( ( the phone ) ( as ( two men ) ) ) ) ) ) ) ) ) ( to ( ( the ( side pain ) ) ( on ( the side ) ) ) ) ) ) . )',
  'sentence2_binary_parse': '( ( ( a guy ) ( near ( a building ) ) ) ( stands ( by ( two ( sames men ) ) ) ) )',
  'orig_sentence1': 'A man standing in front of a building on the phone as two men to the side pain on the side .',
  'orig_sentence2': 'a guy near a building stands by two other men',
  'orig_sentence1_binary_parse': '( ( ( A man ) ( ( standing ( in ( front ( of ( ( a building ) ( on ( ( the phone ) ( as ( two men ) ) ) ) ) ) ) ) ) ( to ( ( the ( side pain ) ) ( on ( the side ) ) ) ) ) ) . )',
  'orig_sentence2_binary_parse': '( ( ( a guy ) ( near ( a building ) ) ) ( sta